# REDUCCIÓN DE LA DIMENSIONALIDAD
<br>

Muchos problemas de aprendizaje automático implican miles o incluso millones de características para cada instancia de entrenamiento. Todas estas características no solo hacen que el entrenamiento sea extremadamente lento, sino que también pueden hacer que sea mucho más difícil encontrar una buena solución, como verás. Este problema a menudo se conoce como la maldición de la dimensionalidad.

Afortunadamente, en los problemas del mundo real, a menudo es posible reducir considerablemente el número de características, convirtiendo un problema intratable en uno manejable. Por ejemplo, considere las imágenes MNIST (introducidas en el Capítulo 3): los píxeles en los bordes de la imagen son casi siempre blancos, por lo que podría soltar por completo estos píxeles del conjunto de entrenamiento sin perder mucha información. Como vimos en el capítulo anterior, (Figura 7-6) confirma que estos píxeles no son en absoluto importantes para la tarea de clasificación. Además, dos píxeles vecinos a menudo están muy correlacionados: si los fusionas en un solo píxel (por ejemplo, tomando la media de las dos intensidades de píxeles), no perderás mucha información.

##### ADVERTENCIA

La reducción de la dimensionalidad causa cierta pérdida de información, al igual que comprimir una imagen a JPEG puede degradar su calidad, por lo que a pesar de que acelerará el entrenamiento, puede hacer que su sistema empeore un poco. También hace que sus tuberías sean un poco más complejas y, por lo tanto, más difíciles de mantener. Por lo tanto, le recomiendo que primero intente entrenar su sistema con los datos originales antes de considerar el uso de la reducción de dimensionalidad. En algunos casos, reducir la dimensionalidad de los datos de entrenamiento puede filtrar algo de ruido y detalles innecesarios y, por lo tanto, resultar en un mayor rendimiento, pero en general no lo hará; solo acelerará el entrenamiento.

##### ----

Además de acelerar la formación, la reducción de la dimensionalidad también es extremadamente útil para la visualización de datos. Reducir el número de dimensiones a dos (o tres) hace posible trazar una vista condensada de un conjunto de entrenamiento de alta dimensión en un gráfico y, a menudo, obtener algunas ideas importantes mediante la detección visual de patrones, como los grupos. Además, la visualización de datos es esencial para comunicar sus conclusiones a las personas que no son científicos de datos, en particular, a los responsables de la toma de decisiones que utilizarán sus resultados.

En este capítulo, primero discutiremos la maldición de la dimensionalidad y tendremos una idea de lo que sucede en el espacio de alta dimensión. Luego consideraremos los dos enfoques principales para la reducción de la dimensionalidad (proyección y aprendizaje múltiple), y pasemos por tres de las técnicas de reducción de dimensionalidad más populares: PCA, proyección aleatoria e incrustación lineal local (LLE).

# La maldición de la dimensionalidad

Estamos tan acostumbrados a vivir en tres dimensiones⁠1 que nuestra intuición nos falla cuando tratamos de imaginar un espacio de alta dimensión. Incluso un hipercubo 4D básico es increíblemente difícil de imaginar en nuestras mentes (ver Figura 8-1), por no hablar de un elipsoide de 200 dimensiones doblado en un espacio de 1.000 dimensiones.

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_0801.png)

(_Figura 8-1. Punto, segmento, cuadrado, cubo y teseracto (hipercubos de 0D a 4D)_)

Resulta que muchas cosas se comportan de manera muy diferente en el espacio de alta dimensión. Por ejemplo, si elige un punto aleatorio en un cuadrado unitario (un cuadrado de 1 × 1), solo tendrá aproximadamente un 0,4 % de probabilidad de estar ubicado a menos de 0,001 desde un borde (en otras palabras, es muy poco probable que un punto aleatorio sea "extremo" a lo largo de cualquier dimensión). Pero en un hipercubo de unidad de 10.000 dimensiones, esta probabilidad es mayor que el 99,9999%. La mayoría de los puntos en un hipercubo de alta dimensión están muy cerca de la frontera.

Aquí hay una diferencia más problemática: si eliges dos puntos al azar en una unidad cuadrada, la distancia entre estos dos puntos será, en promedio, de aproximadamente 0,52. Si eliges dos puntos aleatorios en un cubo de unidades 3D, la distancia media será de aproximadamente 0,66. Pero, ¿qué pasa con dos puntos elegidos al azar en un hipercubo de unidades de 1 000 000 de dimensiones? La distancia promedio, lo creas o no, será de aproximadamente 408,25 (aproximadamente
**sqrt(1000000/6)**) Esto es contradictorio: ¿cómo pueden estar dos puntos tan separados cuando ambos se encuentran dentro del mismo hipercubo de la unidad? Bueno, hay mucho espacio en grandes dimensiones. Como resultado, los conjuntos de datos de alta dimensión corren el riesgo de ser muy escasos: es probable que la mayoría de los casos de entrenamiento estén lejos el uno del otro. Esto también significa que una nueva instancia probablemente estará lejos de cualquier instancia de entrenamiento, lo que hará que las predicciones sean mucho menos confiables que en las dimensiones más bajas, ya que se basarán en extrapolaciones mucho más grandes. En resumen, cuantas más dimensiones tenga el conjunto de entrenamiento, mayor será el riesgo de sobreadaptarlo.

En teoría, una solución a la maldición de la dimensionalidad podría ser aumentar el tamaño del conjunto de entrenamiento para alcanzar una densidad suficiente de instancias de entrenamiento. Desafortunadamente, en la práctica, el número de instancias de entrenamiento necesarias para alcanzar una densidad dada crece exponencialmente con el número de dimensiones. Con solo 100 características, significativamente menos que en el problema del MNIST, todas que van de 0 a 1, necesitarías más instancias de entrenamiento que átomos en el universo observable para que las instancias de entrenamiento estén dentro de 0,1 entre sí en promedio, suponiendo que estuvieran distribuidas uniformemente en todas las dimensiones.

## Enfoques principales para la reducción de la dimensionalidad

Antes de sumergirnos en algoritmos específicos de reducción de dimensionalidad, echemos un vistazo a los dos enfoques principales para reducir la dimensionalidad: la proyección y el aprendizaje múltiple.

### PROYECCIÓN

En la mayoría de los problemas del mundo real, las instancias de entrenamiento no se distribuyen uniformemente en todas las dimensiones. Muchas características son casi constantes, mientras que otras están altamente correlacionadas (como se discutió anteriormente para MNIST). Como resultado, todas las instancias de entrenamiento se encuentran dentro (o cerca de) un subespacio de dimensión mucho menor del espacio de alta dimensión. Esto suena muy abstracto, así que veamos un ejemplo. En la figura 8-2 se puede ver un conjunto de datos 3D representado por pequeñas esferas.

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_0802.png)

(_Figura 8-2. Un conjunto de datos 3D que se encuentra cerca de un subespacio 2D_)

Tenga en cuenta que todas las instancias de entrenamiento se encuentran cerca de un plano: este es un subespacio de dimensión inferior (2D) del espacio de dimensión superior (3D). Si proyectamos cada instancia de entrenamiento perpendicularmente en este subespacio (representado por las líneas discontinuas cortas que conectan las instancias con el plano), obteneremos el nuevo conjunto de datos 2D que se muestra en la Figura 8-3. ¡Ta-da! Acabamos de reducir la dimensionalidad del conjunto de datos de 3D a 2D. Tenga en cuenta que los ejes corresponden a las nuevas características z1 y z2: son las coordenadas de las proyecciones en el plano.

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_0803.png)

(_Figura 8-3. El nuevo conjunto de datos 2D después de la proyección_)

## Aprendizaje múltiple

Sin embargo, la proyección no siempre es el mejor enfoque para la reducción de la dimensionalidad. En muchos casos, el subespacio puede torcerse y girar, como en el famoso conjunto de datos de juguetes de rollo suizo representado en la Figura 8-4.

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_0804.png)

(_Figura 8-4. Conjunto de datos de Swiss Roll_)

Simplemente proyectar en un plano (por ejemplo, dejando caer x3) aplastaría diferentes capas del rollo suizo, como se muestra en el lado izquierdo de la Figura 8-5. Lo que probablemente quieras en su lugar es desenrollar el rollo suizo para obtener el conjunto de datos 2D en el lado derecho de la Figura 8-5.

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_0805.png)

(_Figura 8-5. Aplastar proyectando en un avión (izquierda) frente a desenrollar el rollo suizo (derecha)_)

El rollo suizo es un ejemplo de un colector 2D. En pocas palabras, un colector 2D es una forma 2D que se puede doblar y torcer en un espacio de mayor dimensión. De manera más general, una variedad d-dimensional es una parte de un espacio n-dimensional (donde d < n) que se asemeja localmente a un hiperplano d-dimensional. En el caso del rollo suizo, d = 2 y n = 3: localmente se asemeja a un plano 2D, pero se enrolla en la tercera dimensión.

Muchos algoritmos de reducción de dimensionalidad funcionan modelando la variedad sobre la que se encuentran las instancias de entrenamiento; esto se llama aprendizaje múltiple. Se basa en la suposición de la variedad, también llamada hipótesis de la variedad, que sostiene que la mayoría de los conjuntos de datos de alta dimensión del mundo real se encuentran cerca de una variedad de dimensión mucho más baja. Esta suposición se observa muy a menudo empíricamente.

Una vez más, piense en el conjunto de datos del MNIST: todas las imágenes de dígitos escritas a mano tienen algunas similitudes. Están hechos de líneas conectadas, los bordes son blancos y están más o menos centrados. Si generas imágenes al azar, solo una fracción ridículamente pequeña de ellas se vería como dígitos escritos a mano. En otras palabras, los grados de libertad disponibles para usted si intenta crear una imagen de dígitos son dramáticamente más bajos que los grados de libertad que tiene si se le permite generar cualquier imagen que desee. Estas restricciones tienden a exprimir el conjunto de datos en una variedad de menor dimensión.

La suposición de la variedad a menudo va acompañada de otra suposición implícita: que la tarea en cuestión (por ejemplo, clasificación o regresión) será más simple si se expresa en el espacio de menor dimensión de la variedad. Por ejemplo, en la fila superior de la Figura 8-6, el rollo suizo se divide en dos clases: en el espacio 3D (a la izquierda) el límite de decisión sería bastante complejo, pero en el espacio múltiple desenrollado en 2D (a la derecha) el límite de decisión es una línea recta.

Sin embargo, esta suposición implícita no siempre se mantiene. Por ejemplo, en la fila inferior de la Figura 8-6, el límite de decisión se encuentra en x1 = 5. Este límite de decisión parece muy simple en el espacio 3D original (un plano vertical), pero parece más complejo en el colector desenrollado (una colección de cuatro segmentos de línea independientes).

En resumen, reducir la dimensionalidad de su conjunto de entrenamiento antes de entrenar un modelo generalmente acelerará el entrenamiento, pero puede que no siempre conduzca a una solución mejor o más simple; todo depende del conjunto de datos.

Esperemos que ahora tengas una buena idea de lo que es la maldición de la dimensionalidad y cómo los algoritmos de reducción de la dimensionalidad pueden combatirla, especialmente cuando se cumple la suposición múltiple. El resto de este capítulo pasará por algunos de los algoritmos más populares para la reducción de la dimensionalidad.

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_0806.png)

(_Figura 8-6. El límite de decisión no siempre puede ser más simple con dimensiones más bajas_)

## PCA

El análisis de componentes principales (PCA) es, con mucho, el algoritmo de reducción de dimensionalidad más popular. Primero identifica el hiperplano que se encuentra más cerca de los datos, y luego proyecta los datos sobre él, al igual que en la Figura 8-2.

### Preservando la variedad

Antes de poder proyectar el conjunto de entrenamiento en un hiperplano de dimensión inferior, primero debe elegir el hiperplano correcto. Por ejemplo, un simple conjunto de datos 2D se representa a la izquierda en la Figura 8-7, junto con tres ejes diferentes (es decir, hiperplanos 1D). A la derecha está el resultado de la proyección del conjunto de datos en cada uno de estos ejes. Como puede ver, la proyección sobre la línea sólida conserva la varianza máxima (arriba), mientras que la proyección sobre la línea de puntos conserva muy poca varianza (abajo) y la proyección sobre la línea discontinua conserva una cantidad intermedia de varianza (medio).

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_0807.png)

(_Figura 8-7. Seleccionando el subespacio en el que proyectar_)

Parece razonable seleccionar el eje que preserva la cantidad máxima de varianza, ya que lo más probable es que pierda menos información que las otras proyecciones. Otra forma de justificar esta elección es que es el eje el que minimiza la distancia media al cuadrado entre el conjunto de datos original y su proyección en ese eje. Esta es la idea bastante simple detrás de PCA.

### Componentes principales

PCA identifica el eje que representa la mayor cantidad de variación en el conjunto de entrenamiento. En la Figura 8-7, es la línea sólida. También encuentra un segundo eje, ortogonal al primero, que representa la mayor cantidad de la varianza restante. En este ejemplo 2D no hay otra opción: es la línea de puntos. Si se trata de un conjunto de datos de dimensiones más altas, el PCA también encontraría un tercer eje, ortogonal a ambos ejes anteriores, y un cuarto, un quinto, y así sucesivamente, tantos ejes como el número de dimensiones en el conjunto de datos.

El **i-eje** se llama el i-o componente principal (PC) de los datos. En la Figura 8-7, el primer PC es el eje en el que se encuentra el vectorc1, y el segundo PC es el eje en el que se encuentra el vector **c2**. En la Figura 8-2, los dos primeros PC están en el plano de proyección, y el tercer PC es el eje ortogonal a ese plano. Después de la proyección, en la Figura 8-3, el primer PC corresponde al eje z1, y el segundo PC corresponde al eje z2.

##### NOTA
Para cada componente principal, el PCA encuentra un vector de unidad centrado en cero que apunta en la dirección del PC. Dado que dos vectores de unidad opuestos se encuentran en el mismo eje, la dirección de los vectores de unidad devueltos por el PCA no es estable: si perturba ligeramente el conjunto de entrenamiento y ejecuta el PCA de nuevo, los vectores de unidad pueden apuntar en la dirección opuesta a los vectores originales. Sin embargo, generalmente todavía estarán en los mismos ejes. En algunos casos, un par de vectores unitarios puede incluso girar o intercambiar (si las variaciones a lo largo de estos dos ejes son muy cercanas), pero el plano que definen generalmente seguirá siendo el mismo.
##### ----

Entonces, ¿cómo puedes encontrar los componentes principales de un conjunto de entrenamiento? Afortunadamente, hay una técnica estándar de factorización de matriz llamada descomposición de valor singular (SVD) que puede descomponer la matriz del conjunto de entrenamiento **X** en la multiplicación de la matriz de tres matrices **U Σ V⊺**, donde **V** contiene los vectores unitarios que definen todos los componentes principales que está buscando, como se muestra en la ecuación 8-1.


### Ecuación 8-1. Matriz de componentes principales

<a href="https://imgbb.com/"><img src="https://i.ibb.co/hLkxhqH/Captura-de-pantalla-2023-10-26-a-las-6-16-21.png" alt="Captura-de-pantalla-2023-10-26-a-las-6-16-21" border="0"></a>

The following Python code uses NumPy’s `svd()` function to obtain all the principal components of the 3D training set represented in Figure 8-2, then it extracts the two unit vectors that define the first two PCs:

In [1]:
import numpy as np

#X = [...]  # crear un dataset pequeño 3D
X = np.array([[1, 2, 3],
              [4, 5, 6],
              [7, 8, 9]])
X_centered = X - X.mean(axis=0)
U, s, Vt = np.linalg.svd(X_centered)
c1 = Vt[0]
c2 = Vt[1]

##### ADVERTENCIA

PCA asume que el conjunto de datos se centra en el origen. Como verá, las clases de PCA de Scikit-Learn se encargan de centrar los datos por usted. Si implementa PCA usted mismo (como en el ejemplo anterior), o si utiliza otras bibliotecas, no olvide centrar los datos primero.

##### ----

### Proyección hacia abajo a d Dimensiones

Una vez que haya identificado todos los componentes principales, puede reducir la dimensionalidad del conjunto de datos hacia abajo en las dimensiones dis proyectándolo en el hiperplano definido por los primeros componentes principales d. La selección de este hiperplano garantiza que la proyección conserve la mayor varianza posible. Por ejemplo, en la Figura 8-2, el conjunto de datos 3D se proyecta hacia el plano 2D definido por los dos primeros componentes principales, preservando una gran parte de la variación del conjunto de datos. Como resultado, la proyección 2D se parece mucho al conjunto de datos 3D original.

Para proyectar el conjunto de entrenamiento en el hiperplano y obtener un conjunto de datos reducido **Xd-proj** de dimensionalidad d, calcule la multiplicación de la matriz del conjunto de entrenamiento matriz **X** por la matriz **Wd**, definida como la matriz que contiene las primeras columnas d de **V**, como se muestra en la ecuación 8-2.

### Ecuación 8-2. Proyección de la formación establecida en dimensiones _d_

<a href="https://imgbb.com/"><img src="https://i.ibb.co/JvDQNsk/Captura-de-pantalla-2023-10-29-a-las-5-06-33.png" alt="Captura-de-pantalla-2023-10-29-a-las-5-06-33" border="0"></a><br /><a target='_blank' href='https://es.imgbb.com/'>tipo</a><br/>

El siguiente código Python proyecta el conjunto de entrenamiento en el plano definido por los dos primeros componentes principales:

In [2]:
W2 = Vt[:2].T
X2D = X_centered @ W2

¡Ahí lo tienes! Ahora sabes cómo reducir la dimensionalidad de cualquier conjunto de datos proyectándolo hasta cualquier número de dimensiones, preservando la mayor varianza posible.

## Uso de Scikit-Learn


La clase PCA de Scikit-Learn utiliza SVD para implementar PCA, tal como lo hicimos anteriormente en este capítulo. El siguiente código aplica PCA para reducir la dimensionalidad del conjunto de datos a dos dimensiones (tenga en cuenta que automáticamente se encarga de centrar los datos):

In [3]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X2D = pca.fit_transform(X)

Después de ajustar el transformador PCA al conjunto de datos, su atributo componentes_ contiene la transpuesta de **Wd**: contiene una fila para cada uno de los primeros d componentes principales.


## Relación de varianza explicada

Otra información útil es la relación de varianza explicada de cada componente principal, disponible a través de la variable `explained_variance_ratio_`. 

La relación indica la proporción de la varianza del conjunto de datos que se encuentra a lo largo de cada componente principal. Por ejemplo, echemos un vistazo a las relaciones de varianza explicadas de los dos primeros componentes del conjunto de datos 3D representados en la Figura 8-2:

In [4]:
pca.explained_variance_ratio_

array([1.00000000e+00, 1.44470466e-34])

**(Para el set de datos original, este no es el caso)**

Esta salida nos dice que alrededor del 76 % de la variación del conjunto de datos se encuentra a lo largo del primer PC, y alrededor del 15 % se encuentra a lo largo del segundo PC. Esto deja alrededor del 9 % para el tercer PC, por lo que es razonable suponer que el tercer PC probablemente tenga poca información.

## Elegir el número correcto de dimensiones

En lugar de elegir arbitrariamente el número de dimensiones a reducir, es más simple elegir el número de dimensiones que suman una parte suficientemente grande de la varianza, digamos, el 95 % (una excepción a esta regla, por supuesto, es si está reduciendo la dimensionalidad para la visualización de datos, en cuyo caso querrá reducir la dimensionalidad a 2 o 3).

El siguiente código carga y divide el conjunto de datos MNIST (introducido en el Capítulo 3) y realiza PCA sin reducir la dimensionalidad, luego calcula el número mínimo de dimensiones necesarias para preservar el 95 % de la varianza del conjunto de entrenamiento:

In [5]:
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', as_frame=False)
X_train, y_train = mnist.data[:60_000], mnist.target[:60_000]
X_test, y_test = mnist.data[60_000:], mnist.target[60_000:]

pca = PCA()
pca.fit(X_train)
cumsum = np.cumsum(pca.explained_variance_ratio_)
d = np.argmax(cumsum >= 0.95) + 1  # d equals 154

Luego podría configurar `n_components=d` y ejecutar PCA nuevamente, pero hay una opción mejor. En lugar de especificar el número de componentes principales que desea conservar, puede configurar `n_components` para que sea un valor flotante entre 0,0 y 1,0, lo que indica la proporción de varianza que desea conservar:

In [6]:
pca = PCA(n_components=0.95)
X_reduced = pca.fit_transform(X_train)

El número real de componentes se determina durante el entrenamiento y se almacena en el atributo `n_components_`:

In [7]:
pca.n_components_

154

Otra opción es trazar la varianza explicada en función del número de dimensiones (simplemente trazar `cumsum`; ver Figura 8-8). Por lo general, habrá un codo en la curva, donde la varianza explicada deja de crecer rápidamente. En este caso, se puede ver que reducir la dimensionalidad a unas 100 dimensiones no perdería demasiada varianza explicada.

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_0808.png)

(_Figura 8-8. Explicación de la varianza en función del número de dimensiones_)

Por último, si está utilizando la reducción de dimensionalidad como paso de preprocesamiento para una tarea de aprendizaje supervisado (por ejemplo, clasificación), puede ajustar el número de dimensiones como lo haría con cualquier otro hiperparámetro (consulte el Capítulo 2). Por ejemplo, el siguiente ejemplo de código crea una canalización de dos pasos: primero reduce la dimensionalidad mediante PCA y luego clasifica mediante un bosque aleatorio. A continuación, utiliza `RandomizedSearchCV` para encontrar una buena combinación de hiperparámetros tanto para PCA como para el clasificador de bosque aleatorio. Este ejemplo realiza una búsqueda rápida, ajusta solo 2 hiperparámetros, entrena en solo 1000 instancias y ejecuta solo 10 iteraciones, pero siéntete libre de hacer una búsqueda más exhaustiva si tienes tiempo:

In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import make_pipeline

clf = make_pipeline(PCA(random_state=42),
                    RandomForestClassifier(random_state=42))
param_distrib = {
    "pca__n_components": np.arange(10, 80),
    "randomforestclassifier__n_estimators": np.arange(50, 500)
}
rnd_search = RandomizedSearchCV(clf, param_distrib, n_iter=10, cv=3,
                                random_state=42)
rnd_search.fit(X_train[:1000], y_train[:1000])

RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('pca', PCA(random_state=42)),
                                             ('randomforestclassifier',
                                              RandomForestClassifier(random_state=42))]),
                   param_distributions={'pca__n_components': array([10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26,
       27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43,
       44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60,
       6...
       414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426,
       427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439,
       440, 441, 442, 443, 444, 445, 446, 447, 448, 449, 450, 451, 452,
       453, 454, 455, 456, 457, 458, 459, 460, 461, 462, 463, 464, 465,
       466, 467, 468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478,
       479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 491,
       

In [9]:
# Echemos un vistazo a los mejores hiperparámetros encontrados:

print(rnd_search.best_params_)

{'randomforestclassifier__n_estimators': 465, 'pca__n_components': 23}


Es interesante observar lo bajo que es el número óptimo de componentes: ¡reducimos un conjunto de datos de 784 dimensiones a solo 23 dimensiones! Esto está relacionado con el hecho de que usamos un bosque aleatorio, que es un modelo bastante poderoso. Si usáramos un modelo lineal en su lugar, como un `SGDClassifier`, la búsqueda encontraría que necesitamos preservar más dimensiones (alrededor de 70).

## PCA para comprensión

Después de la reducción de la dimensionalidad, el conjunto de entrenamiento ocupa mucho menos espacio. Por ejemplo, después de aplicar PCA al conjunto de datos del MNIST mientras se preserva el 95 % de su variación, nos quedamos con 154 características, en lugar de las 784 características originales. Así que el conjunto de datos es ahora menos del 20 % de su tamaño original, ¡y solo perdimos el 5 % de su variación! Esta es una relación de compresión razonable, y es fácil ver cómo tal reducción de tamaño aceleraría enormemente un algoritmo de clasificación.

También es posible descomprimir el conjunto de datos reducido de nuevo a 784 dimensiones aplicando la transformación inversa de la proyección de PCA. Esto no le devolverá los datos originales, ya que la proyección perdió un poco de información (dentro de la variación del 5% que se eliminó), pero es probable que esté cerca de los datos originales. La distancia media al cuadrado entre los datos originales y los datos reconstruido (comprimidos y luego descomprimidos) se llama error de reconstrucción.

El método `inverse_transform()` nos permite descomprimir el conjunto de datos MNIST reducido a 784 dimensiones:

In [10]:
X_recovered = pca.inverse_transform(X_reduced)

La figura 8-9 muestra algunos dígitos del conjunto de entrenamiento original (a la izquierda) y los dígitos correspondientes después de la compresión y la descompresión. Puedes ver que hay una ligera pérdida de calidad de imagen, pero los dígitos siguen intactos en su mayoría.

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_0809.png)

(_Figura 8-9. Compresión MNIST que conserva el 95 % de la variación_)

La ecuación para la transformación inversa se muestra en la ecuación 8-3.

### Ecuación 8-3. Transformación inversa de PCA, de vuelta al número original de dimensiones

<a href="https://imgbb.com/"><img src="https://i.ibb.co/KscF01y/Captura-de-pantalla-2023-10-29-a-las-6-05-39.png" alt="Captura-de-pantalla-2023-10-29-a-las-6-05-39" border="0"></a>


## PCA al azar (random)

Si configura el hiperparámetro svd_solver en `"randomized"`, Scikit-Learn utiliza un algoritmo estocástico llamado PCA aleatorio que encuentra rápidamente una aproximación de los primeros d componentes principales. Su complejidad computacional es O(m × d2) + O(d3), en lugar de O(m × n2) + O(n3) para el enfoque SVD completo, por lo que es dramáticamente más rápido que el SVD completo cuando d es mucho menor que n:


In [ ]:
rnd_pca = PCA(n_components=154, svd_solver="randomized", random_state=42)
X_reduced = rnd_pca.fit_transform(X_train)

#### TIP

De forma predeterminada, svd_solver en realidad está configurado en `"auto"`: Scikit-Learn usa automáticamente el algoritmo PCA aleatorio si max(m, n) > 500 y n_components es un número entero menor que el 80% de min(m, n), o de lo contrario utiliza el enfoque SVD completo. Por lo tanto, el código anterior usaría el algoritmo PCA aleatorio incluso si eliminara el argumento `svd_solver="randomized"`, ya que 154 < 0,8 × 784. Si desea forzar a Scikit-Learn a usar SVD completo para obtener un resultado un poco más preciso, puede establezca el hiperparámetro svd_solver en "full".

#### ----

## PCA incremental

Un problema con las implementaciones anteriores de PCA es que requieren que todo el conjunto de entrenamiento quepa en la memoria para que el algoritmo se ejecute. Afortunadamente, se han desarrollado algoritmos incrementales de PCA (IPCA) que le permiten dividir el conjunto de entrenamiento en mini-lotes y alimentarlos en un mini-lote a la vez. Esto es útil para grandes conjuntos de entrenamiento y para aplicar PCA en línea (es decir, sobre la marcha, a medida que llegan nuevas instancias).

El siguiente código divide el conjunto de entrenamiento MNIST en 100 minilotes (usando la función `array_split()` de NumPy) y los envía a la clase `IncrementalPCA⁠` de Scikit-Learn para reducir la dimensionalidad del conjunto de datos MNIST a 154 dimensiones, como antes. Tenga en cuenta que debe llamar al método `partial_fit()` con cada mini-lote, en lugar del método `fit()` con todo el conjunto de entrenamiento:

In [ ]:
from sklearn.decomposition import IncrementalPCA

n_batches = 100
inc_pca = IncrementalPCA(n_components=154)
for X_batch in np.array_split(X_train, n_batches):
    inc_pca.partial_fit(X_batch)

X_reduced = inc_pca.transform(X_train)

Alternativamente, puede usar la clase `memmap` de NumPy, que le permite manipular una gran matriz almacenada en un archivo binario en el disco como si estuviera completamente en la memoria; la clase carga solo los datos que necesita en la memoria, cuando los necesita. Para demostrar esto, primero creemos un archivo mapeado en memoria (memmap) y copiemos en él el conjunto de entrenamiento MNIST, luego llamemos a `flush()` para asegurarnos de que cualquier dato que aún esté en el caché se guarde en el disco. En la vida real, `X_train` normalmente no cabe en la memoria, por lo que lo cargarías fragmento por fragmento y guardarías cada fragmento en la parte derecha de la matriz memmap:

In [ ]:
filename = "my_mnist.mmap"
X_mmap = np.memmap(filename, dtype='float32', mode='write', shape=X_train.shape)
X_mmap[:] = X_train  # could be a loop instead, saving the data chunk by chunk
X_mmap.flush()

A continuación, podemos cargar el archivo memmap y usarlo como una matriz NumPy normal. Usemos la clase `IncrementalPCA` para reducir su dimensionalidad. Dado que este algoritmo utiliza sólo una pequeña parte de la matriz en un momento dado, el uso de la memoria permanece bajo control. Esto hace posible llamar al método `fit()` habitual en lugar de `partial_fit()`, lo cual es bastante conveniente:

In [ ]:
X_mmap = np.memmap(filename, dtype="float32", mode="readonly").reshape(-1, 784)
batch_size = X_mmap.shape[0] // n_batches
inc_pca = IncrementalPCA(n_components=154, batch_size=batch_size)
inc_pca.fit(X_mmap)

#### ADVERTENCIA

Sólo los datos binarios sin procesar se guardan en el disco, por lo que debe especificar el tipo de datos y la forma de la matriz cuando la carga. Si omite la forma, `np.memmap()` devuelve una matriz 1D.

#### ---

Para conjuntos de datos de dimensiones muy altas, el PCA puede ser demasiado lento. Como vio anteriormente, incluso si utiliza PCA aleatoria, su complejidad computacional sigue siendo O(m × d2) + O(d3), por lo que el número objetivo de dimensiones d no debe ser demasiado grande. Si se trata de un conjunto de datos con decenas de miles de características o más (por ejemplo, imágenes), entonces el entrenamiento puede ser demasiado lento: en este caso, debería considerar el uso de una proyección aleatoria en su lugar.

## Proyección aleatoria


Como su nombre indica, el algoritmo de proyección aleatoria proyecta los datos en un espacio de menor dimensión utilizando una proyección lineal aleatoria. Esto puede parecer una locura, pero resulta que es muy probable que una proyección aleatoria de este tipo preserve las distancias bastante bien, como demostró matemáticamente William B. Johnson y Joram Lindenstrauss en un lema famoso. Por lo tanto, dos instancias similares seguirán siendo similares después de la proyección, y dos instancias muy diferentes seguirán siendo muy diferentes.

Obviamente, cuantas más dimensiones dejes caer, más información se pierde y más distancias se distorsionan. Entonces, ¿cómo puedes elegir el número óptimo de dimensiones? Bueno, a Johnson y Lindenstrauss se les ocurrió una ecuación que determina el número mínimo de dimensiones a preservar para garantizar, con alta probabilidad, que las distancias no cambien en más de una tolerancia dada. Por ejemplo, si tiene un conjunto de datos que contiene m = 5.000 instancias con n = 20.000 características cada una, y no desea que la distancia cuadrada entre dos instancias cualquiera cambie en más de ε = 10%,6, entonces debe proyectar los datos hasta d dimensiones, con d ≥ 4 log(m) / (½ ε² - 1⁄3 ε³), que es 7.300 dimensiones. ¡Esa es una reducción de dimensionalidad bastante significativa! Tenga en cuenta que la ecuación no utiliza n, solo se basa en m y ε. Esta ecuación está implementada por la función `johnson_lindenstrauss_min_dim()`:

In [ ]:
from sklearn.random_projection import johnson_lindenstrauss_min_dim
m, ε = 5_000, 0.1
d = johnson_lindenstrauss_min_dim(m, eps=ε)
d

Ahora solo podemos generar una matriz aleatoria **P** de forma [d, n], donde cada elemento se muestrea al azar de una distribución gaussiana con media 0 y varianza 1 / d, y usarla para proyectar un conjunto de datos desde n dimensiones hasta d:

In [ ]:
n = 20_000
np.random.seed(42)
P = np.random.randn(d, n) / np.sqrt(d)  # std dev = raíz cuadrada de la varianza

X = np.random.randn(m, n)  # genera dataset falso
X_reduced = X @ P.T

¡Eso es todo! Es simple y eficiente, y no se requiere entrenamiento: lo único que el algoritmo necesita para crear la matriz aleatoria es la forma del conjunto de datos. Los datos en sí no se utilizan en absoluto.

Scikit-Learn ofrece una clase `GaussianRandomProjection` para hacer exactamente lo que acabamos de hacer: cuando llamas a su método `fit()`, usa `johnson_lindenstrauss_min_dim()` para determinar la dimensionalidad de salida, luego genera una matriz aleatoria, que almacena en el atributo `components_`. 

Luego, cuando llamas a `transform()`, utiliza esta matriz para realizar la proyección. Al crear el transformador, puede configurar eps si desea modificar **ε** (el valor predeterminado es 0.1) y `n_components` si desea forzar una dimensionalidad objetivo específica d. 

El siguiente ejemplo de código proporciona el mismo resultado que el código anterior (también puede verificar que `gaussian_rnd_proj.components_` es igual a `P`):

In [ ]:
from sklearn.random_projection import GaussianRandomProjection

gaussian_rnd_proj = GaussianRandomProjection(eps=ε, random_state=42)
X_reduced = gaussian_rnd_proj.fit_transform(X)  # mismo resultado que arriba

Scikit-Learn también proporciona un segundo transformador de proyección aleatoria, conocido como `SparseRandomProjection`. Determina la dimensionalidad del objetivo de la misma manera, genera una matriz aleatoria de la misma forma y realiza la proyección de forma idéntica. 
La principal diferencia es que la matriz aleatoria es escasa. 

Esto significa que utiliza mucha menos memoria: ¡unas 25 MB en lugar de casi 1,2 GB en el ejemplo anterior! Y también es mucho más rápido, tanto para generar la matriz aleatoria como para reducir la dimensionalidad: alrededor de un 50 % más rápido en este caso. Además, si la entrada es escasa, la transformación la mantiene escasa (a menos que establezcas `dense_output=True`). 

Por último, disfruta de la misma propiedad de conservación de la distancia que el enfoque anterior, y la calidad de la reducción de la dimensionalidad es comparable. En resumen, generalmente es preferible usar este transformador en lugar del primero, especialmente para conjuntos de datos grandes o escasos.

La relación r de elementos distintos de cero en la matriz aleatoria dispersa se llama su densidad. Por defecto, es igual a **1/sqrt(N)**. 

Con 20.000 funciones, esto significa que sólo 1 de cada 141 celdas de la matriz aleatoria es distinta de cero: ¡eso es bastante escaso! Puede establecer el hiperparámetro de `density` en otro valor si lo prefiere. Cada celda de la matriz aleatoria dispersa tiene una probabilidad r de ser distinta de cero, y cada valor distinto de cero es –v o +v (ambos con la misma probabilidad), donde **v = 1/sqrt(dr)**.

Si desea realizar la transformación inversa, primero debe calcular la pseudoinversa de la matriz de componentes usando la función `pinv()` de SciPy, luego multiplicar los datos reducidos por la transposición de la pseudoinversa:

In [ ]:
components_pinv = np.linalg.pinv(gaussian_rnd_proj.components_)
X_recovered = X_reduced @ components_pinv.T

#### ADVERTENCIA

Calcular la pseudoinversa puede llevar mucho tiempo si la matriz de componentes es grande, ya que la complejidad computacional de pinv() es O(dn²) si d < n, u O(nd²) en caso contrario.

En resumen, la proyección aleatoria es un algoritmo de reducción de dimensionalidad simple, rápido, eficiente en memoria y sorprendentemente potente que debe tener en cuenta, especialmente cuando se trata de conjuntos de datos de alta dimensión.

#### NOTA

La proyección aleatoria no siempre se utiliza para reducir la dimensionalidad de los grandes conjuntos de datos. Por ejemplo, un artículo de 2017⁠ de Sanjoy Dasgupta et al. mostró que el cerebro de una mosca de la fruta implementa un análogo de proyección aleatoria para mapear entradas olfativas densas de baja dimensión a salidas binarias dispersas de alta dimensión: por cada olor, solo una pequeña fracción de las neuronas de salida se activan, pero olores similares activan muchas de las mismas neuronas. 

Esto es similar a un algoritmo bien conocido llamado hash sensible a la localidad (LSH), que normalmente se utiliza en los motores de búsqueda para agrupar documentos similares.

## LLE 
### (Incrustación Lineal Local)

La incrustación lineal local (LLE)⁠ es una técnica de reducción de dimensionalidad no lineal (NLDR). Es una técnica de aprendizaje múltiple que no se basa en las proyecciones, a diferencia de la PCA y la proyección aleatoria. 

En pocas palabras, LLE trabaja midiendo primero cómo cada instancia de entrenamiento se relaciona linealmente con sus vecinos más cercanos, y luego buscando una representación de baja dimensión del conjunto de entrenamiento donde estas rel

aciones locales se conservan mejor (más detalles en breve). Este enfoque lo hace particularmente bueno para desenrollar los colectores retorcidos, especialmente cuando no hay demasiado ruido.

El siguiente código realiza una tirada suiza (Swiss roll) y luego utiliza la clase `LocallyLinearEmbedding` de Scikit-Learn para desenrollarla:

In [ ]:
from sklearn.datasets import make_swiss_roll
from sklearn.manifold import LocallyLinearEmbedding

X_swiss, t = make_swiss_roll(n_samples=1000, noise=0.2, random_state=42)
lle = LocallyLinearEmbedding(n_components=2, n_neighbors=10, random_state=42)
X_unrolled = lle.fit_transform(X_swiss)

La variable `t` es una matriz NumPy 1D que contiene la posición de cada instancia a lo largo del eje enrollado del rollo suizo. No lo usamos en este ejemplo, pero puede usarse como objetivo para una tarea de regresión no lineal.

El conjunto de datos 2D resultante se muestra en la Figura 8-10. Como puede ver, el rollo suizo está completamente desenrollado, y las distancias entre instancias están bien conservadas localmente. Sin embargo, las distancias no se conservan a mayor escala: el rollo suizo desenrollado debe ser un rectángulo, no este tipo de banda estirada y retorcida. Sin embargo, LLE hizo un buen trabajo modelando el colector.

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_0810.png)

__Figura 8-10. Rollo suizo desenrollado usando LLE__

Así es como funciona LLE: para cada instancia de entrenamiento **x(i)**, el algoritmo identifica a sus vecinos más cercanos (en el código anterior k = 10), y luego intenta reconstruir **x(i)** como una función lineal de estos vecinos. Más específicamente, trata de encontrar el weightswi,j de tal manera que la distancia cuadrada entre **x(i)** y **∑(j=1 a m) de wi,j * X^(j)** es lo más pequeño posible, asumiendo que wi,j = 0 si x(j) no es uno de los vecinos más cercanos de x(i). 
Por lo tanto, el primer paso de LLE es el problema de optimización restringida descrito en la ecuación 8-4, donde W es la matriz de peso que contiene todos los pesos wi,j. La segunda restricción simplemente normaliza los pesos para cada instancia de entrenamiento x(i).

### Ecuación 8-4. Paso 1 de LLE: modelado lineal de las relaciones locales

<a href="https://imgbb.com/"><img src="https://i.ibb.co/DgWTFCH/Captura-de-pantalla-2023-11-19-a-las-21-24-50.png" alt="Captura-de-pantalla-2023-11-19-a-las-21-24-50" border="0"></a>

Después de este paso, la matriz de peso **W^**(que contiene los pesos W^i,j) codifica las relaciones lineales locales entre las instancias de entrenamiento. 

El segundo paso es mapear las instancias de entrenamiento en un espacio de dimensión d (donde d < n) preservando estas relaciones locales tanto como sea posible. Si z(i) es la imagen de x(i) en este espacio de dimensión d, entonces queremos la distancia cuadrada entre z(i) y ∑ (j=1 a m) de W^ i,j * Z^(j)
ser lo más pequeño posible. 

Esta idea conduce al problema de optimización sin restricciones descrito en la ecuación 8-5. Se ve muy similar al primer paso, pero en lugar de mantener las instancias fijas y encontrar los pesos óptimos, estamos haciendo lo contrario: mantener los pesos fijos y encontrar la posición óptima de las imágenes de las instancias en el espacio de baja dimensión. Tenga en cuenta que **Z** es la matriz que contiene todos los **z(i)**.

### Ecuación 8-5. Paso 2 de LLE: reducir la dimensionalidad mientras se preservan las relaciones

<a href="https://imgbb.com/"><img src="https://i.ibb.co/drGv03f/Captura-de-pantalla-2023-11-19-a-las-21-28-35.png" alt="Captura-de-pantalla-2023-11-19-a-las-21-28-35" border="0"></a>


La implementación de LLE de Scikit-Learn tiene la siguiente complejidad computacional: O(m log(m)n log(k)) para encontrar los vecinos más cercanos más cercanos, O(mnk3) para optimizar los pesos y O(dm2) para construir las representaciones de baja dimensión. Desafortunadamente, el m2 en el último término hace que este algoritmo escale mal a conjuntos de datos muy grandes.

Como puede ver, el LLE es bastante diferente de las técnicas de proyección, y es significativamente más complejo, pero también puede construir representaciones de baja dimensión mucho mejores, especialmente si los datos no son lineales.

## Otras técnicas de reducción de dimensionalidad


Antes de concluir este capítulo, echemos un vistazo rápido a algunas otras técnicas populares de reducción de dimensionalidad disponibles en Scikit-Learn:

- **`sklearn.manifold.MDS`**

El escalado multidimensional (MDS) reduce la dimensionalidad al tiempo que se trata de preservar las distancias entre las instancias. La proyección aleatoria hace eso para datos de alta dimensión, pero no funciona bien con datos de baja dimensión.

- **`sklearn.manifold.Isomap`**

Isomap crea un gráfico conectando cada instancia con sus vecinos más cercanos, luego reduce la dimensionalidad mientras trata de preservar las distancias geodésicas entre las instancias. La distancia geodésica entre dos nodos en un gráfico es el número de nodos en el camino más corto entre estos nodos.

- **`sklearn.manifold.TSNE`**

La incrustación de vecinos estocásticos distribuidos por t (t-SNE) reduce la dimensionalidad mientras se trata de mantener las instancias similares cerca y las instancias dissimilares separadas. Se utiliza principalmente para la visualización, en particular para visualizar grupos de instancias en el espacio de alta dimensión. Por ejemplo, en los ejercicios al final de este capítulo usarás t-SNE para visualizar un mapa 2D de las imágenes de MNIST.

- **`sklearn.discriminant_analysis.LinearDiscriminantAnalysis`**

El análisis de discriminantes lineales (LDA) es un algoritmo de clasificación lineal que, durante el entrenamiento, aprende los ejes más discriminativos entre las clases. Estos ejes se pueden utilizar para definir un hiperplano en el que proyectar los datos. El beneficio de este enfoque es que la proyección mantendrá las clases lo más separadas posible, por lo que LDA es una buena técnica para reducir la dimensionalidad antes de ejecutar otro algoritmo de clasificación (a menos que LDA por sí sola sea suficiente).

La figura 8-11 muestra los resultados de MDS, Isomap y t-SNE en el rollo suizo. MDS logra aplanar el rollo suizo sin perder su curvatura global, mientras que Isomap lo deja caer por completo. 

Dependiendo de la tarea aguas abajo, preservar la estructura a gran escala puede ser buena o mala. t-SNE hace un trabajo razonable de aplanar el rollo suizo, preservando un poco de curvatura, y también amplifica los grupos, rompiendo el rollo. Una vez más, esto podría ser bueno o malo, dependiendo de la tarea posterior.

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_0811.png)

__Figura 8-11. Usando varias técnicas para reducir el rollo suizo a 2D__

## Ejercicios

1- ¿Cuáles son las principales motivaciones para reducir la dimensionalidad de un conjunto de datos? ¿Cuáles son los principales inconvenientes?

2- ¿Cuál es la maldición de la dimensionalidad?

3- Una vez que se ha reducido la dimensionalidad de un conjunto de datos, ¿es posible revertir la operación? Si es así, ¿cómo? Si no, ¿por qué?

4- ¿Se puede utilizar la PCA para reducir la dimensionalidad de un conjunto de datos altamente no lineal?

5- Supongamos que realiza PCA en un conjunto de datos de 1.000 dimensiones, estableciendo la relación de varianza explicada en el 95 %. ¿Cuántas dimensiones tendrá el conjunto de datos resultante?

6- ¿En qué casos usaría PCA regular, PCA incremental, PCA aleatoria o proyección aleatoria?

7- ¿Cómo puede evaluar el rendimiento de un algoritmo de reducción de dimensionalidad en su conjunto de datos?

8- ¿Tiene algún sentido encadenar dos algoritmos de reducción de dimensionalidad diferentes?

9- Cargue el conjunto de datos MNIST (introducido en el capítulo 3) y divídalo en un conjunto de entrenamiento y un conjunto de pruebas (tome las primeras 60.000 instancias para la formación y las 10.000 restantes para las pruebas). Entrene a un clasificador de bosques al azar en el conjunto de datos y el tiempo que tarda, luego evalúe el modelo resultante en el conjunto de pruebas. A continuación, use PCA para reducir la dimensionalidad del conjunto de datos, con una relación de varianza explicada del 95 %. Entrene a un nuevo clasificador de bosques al azar en el conjunto de datos reducido y vea cuánto tiempo lleva. ¿El entrenamiento fue mucho más rápido? A continuación, evalúe el clasificador en el conjunto de pruebas. ¿Cómo se compara con el clasificador anterior? Inténtalo de nuevo con un SGDClassifier. ¿Cuánto ayuda la PCA ahora?

10- Utilice t-SNE para reducir las primeras 5.000 imágenes del conjunto de datos MNIST a 2 dimensiones y trace el resultado utilizando Matplotlib. Puedes usar un diagrama de dispersión usando 10 colores diferentes para representar la clase objetivo de cada imagen. Alternativamente, puede reemplazar cada punto en el diagrama de dispersión con la clase de la instancia correspondiente (un dígito de 0 a 9), o incluso trazar versiones reducidas de las propias imágenes de dígitos (si traza todos los dígitos, la visualización estará demasiado desordenada, por lo que debe dibujar una muestra aleatoria o trazar una instancia solo si no se ha trazado ninguna otra instancia a una distancia cercana). Deberías obtener una buena visualización con grupos de dígitos bien separados. Intente utilizar otros algoritmos de reducción de dimensionalidad, como PCA, LLE o MDS, y compare las visualizaciones resultantes.